<a href="https://colab.research.google.com/github/oytungunes/ResearchBilkent/blob/main/PeopleWalking_Range_Doppler_BiLSTM_threeclass_noaugment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Burada people walking verisi ile fw, sw, ve swp ile sadece range-doppler 2D image datası biLSTM ile multiclass train edildi.

#Mean test accuracy is 0.604, max test accuracy is 0.626, min test accuracy is 0.567,  std of test accuracy is 0.027.


In [2]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization, Input, Conv2D, MaxPooling2D, Concatenate, GRU, LSTM, GRU, TimeDistributed, Bidirectional
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import EarlyStopping
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K
import gc
import os
from keras.layers import LeakyReLU
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Load Range-Doppler fast data
range_doppler_fast = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_fast.mat')
range_doppler_fast = range_doppler_fast['range_doppler_fast']
range_doppler_fast = np.transpose(range_doppler_fast, (2, 0, 1)) # label x width x height
range_doppler_fast_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_fast_label.mat')
range_doppler_fast_label = range_doppler_fast_label['range_doppler_fast_label']

# Load Range-Doppler slow data

range_doppler_slow = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_slow.mat')
range_doppler_slow = range_doppler_slow['range_doppler_slow']
range_doppler_slow = np.transpose(range_doppler_slow, (2, 0, 1))# label x width x height
range_doppler_slow_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_slow_label.mat')
range_doppler_slow_label = range_doppler_slow_label['range_doppler_slow_label']

# Load Range-Doppler slow with pockets data
range_doppler_swp = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_swp.mat')
range_doppler_swp = range_doppler_swp['range_doppler_swp']
range_doppler_swp = np.transpose(range_doppler_swp, (2, 0, 1))# label x width x height
range_doppler_swp_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_swp_label.mat')
range_doppler_swp_label = range_doppler_swp_label['range_doppler_swp_label']


# Load Spectrogram fast data
spectrogram_fast_resized = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_fast_resized.mat')
spectrogram_fast_resized = spectrogram_fast_resized['spectrogram_fast_resized']
spectrogram_fast_resized = np.transpose(spectrogram_fast_resized, (2, 0, 1)) # label x width x height
spectrogram_fast_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_fast_label.mat')
spectrogram_fast_label = spectrogram_fast_label['spectrogram_fast_label']


# Load Spectrogram slow data
spectrogram_slow_resized= scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_slow_resized.mat')
spectrogram_slow_resized = spectrogram_slow_resized['spectrogram_slow_resized']
spectrogram_slow_resized = np.transpose(spectrogram_slow_resized, (2, 0, 1)) # label x width x height
spectrogram_slow_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_slow_label.mat')
spectrogram_slow_label = spectrogram_slow_label['spectrogram_slow_label']

# Load Spectrogram slow with pockets data
spectrogram_swp = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_swp.mat')
spectrogram_swp = spectrogram_swp['spectrogram_swp']
spectrogram_swp = np.transpose(spectrogram_swp, (2, 0, 1))# label x width x height
spectrogram_swp_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_swp_label.mat')
spectrogram_swp_label = spectrogram_swp_label['spectrogram_swp_label']


In [ ]:
print(spectrogram_fast_resized.shape)
print(spectrogram_slow_resized.shape)
print(spectrogram_swp.shape)
print(spectrogram_fast_label.shape)
print(spectrogram_slow_label.shape)
print(spectrogram_swp_label.shape)


(57, 100, 100)
(57, 100, 100)
(57, 100, 100)
(57, 1)
(57, 1)
(57, 1)


In [6]:
range_doppler_concat_label = np.concatenate((range_doppler_fast_label,range_doppler_slow_label,range_doppler_swp_label),axis=0)
spectrogram_concat_label = np.concatenate((spectrogram_fast_label,spectrogram_slow_label,spectrogram_swp_label),axis=0)

# One-Hot Encode Label Spectrogram
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(spectrogram_concat_label)
# binary encode
onehot_encoder = OneHotEncoder(sparse_output=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
spectrogram_concat_label_onehotencoded = onehot_encoder.fit_transform(integer_encoded)

# One-Hot Encode Label Range-Doppler
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(range_doppler_concat_label)
# binary encode
onehot_encoder = OneHotEncoder(sparse_output=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
range_doppler_concat_label_onehotencoded = onehot_encoder.fit_transform(integer_encoded)



/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
# Concat range-doppler data
range_doppler_concat = np.concatenate((range_doppler_fast,range_doppler_slow,range_doppler_swp),axis=0) # fast =fall, slow=walking
range_doppler_concat = range_doppler_concat[:,:,:,np.newaxis]

# Shuffle concat range doppler
shuffle_indx = random.sample(range(0, range_doppler_concat.shape[0]), range_doppler_concat.shape[0]) # split validation data
range_doppler_concat_shuffle = range_doppler_concat[shuffle_indx,:,:,:]
range_doppler_concat_label_shuffle_onehotencoded = range_doppler_concat_label_onehotencoded[shuffle_indx,:]
range_doppler_concat_label_shuffle = range_doppler_concat_label[shuffle_indx,:]



# Concat spectrogram data

spectrogram_concat = np.concatenate((spectrogram_fast_resized,spectrogram_slow_resized,spectrogram_swp),axis=0)
spectrogram_concat = spectrogram_concat[:,:,:,np.newaxis]

# Shuffle concat spectrogram
spectrogram_concat_shuffle = spectrogram_concat[shuffle_indx,:,:,:]
spectrogram_concat_label_shuffle_onehotencoded = spectrogram_concat_label_onehotencoded[shuffle_indx,:]
spectrogram_concat_label_shuffle = spectrogram_concat_label[shuffle_indx,:]

In [8]:
def split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,label_shuffle_train):

  dummy_label = np.zeros((spectrogram_concat_shuffle_train.shape[0],1))
  for randomlist_for_train_indx, randomlist_for_validation_indx in kfold.split(spectrogram_concat_shuffle_train,dummy_label):
    randomlist_for_validation_indx

  # get validation data
  spectrogram_validation_data = spectrogram_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  range_doppler_validation_data = range_doppler_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]

  # get training data
  spectrogram_training_data = spectrogram_concat_shuffle_train[randomlist_for_train_indx,:,:,:]
  range_doppler_training_data = range_doppler_concat_shuffle_train[randomlist_for_train_indx,:,:,:]


  #get labels
  validation_labels = label_shuffle_train[randomlist_for_validation_indx,:]
  training_labels = label_shuffle_train[randomlist_for_train_indx,:]


  # get CLASS1 and CLASS2 indexes of training dat

  class1_indexes = np.where(training_labels == 0)[0]
  class2_indexes = np.where(training_labels == 1)[0]
  class3_indexes = np.where(training_labels == 2)[0]


  class1_spectrograms_train = spectrogram_training_data[class1_indexes,:,:,:]
  class2_spectrograms_train = spectrogram_training_data[class2_indexes,:,:,:]
  class3_spectrograms_train = spectrogram_training_data[class3_indexes,:,:,:]


  class1_rangedoppler_train = range_doppler_training_data[class1_indexes,:,:,:]
  class2_rangedoppler_train = range_doppler_training_data[class2_indexes,:,:,:]
  class3_rangedoppler_train = range_doppler_training_data[class3_indexes,:,:,:]


  size_of_samples_class1= class1_spectrograms_train.shape[0]
  size_of_samples_class2 = class2_spectrograms_train.shape[0]
  size_of_samples_class3 = class3_spectrograms_train.shape[0]


  class1_label = np.zeros((size_of_samples_class1,1))
  class2_label = np.ones((size_of_samples_class2,1))
  class3_label = 2*np.ones((size_of_samples_class3,1))


  # unify data
  spectrogram_training_data = np.concatenate((class3_spectrograms_train,class2_spectrograms_train,class1_spectrograms_train),axis=0)
  range_doppler_training_data = np.concatenate((class3_rangedoppler_train,class2_rangedoppler_train,class1_rangedoppler_train),axis=0)


  training_labels = np.concatenate((class3_label,class2_label,class1_label),axis=0)


  return (spectrogram_training_data,range_doppler_training_data,training_labels,\
     spectrogram_validation_data,range_doppler_validation_data, validation_labels)


In [9]:
def normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable):
  # ---------------- Normalize Inputs ----------------
  if normalize_inputs_enable == True:
    layer = Normalization(axis=None)
    layer.adapt(range_doppler_concat_shuffle)
    range_doppler_concat_shuffle = layer(range_doppler_concat_shuffle)
  else:
    range_doppler_concat_shuffle = range_doppler_concat_shuffle
  return(range_doppler_concat_shuffle)


normalize_inputs_enable = 1
range_doppler_concat_shuffle = normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable)
spectrogram_concat_shuffle = normalize_inputs(spectrogram_concat_shuffle, normalize_inputs_enable)
range_doppler_concat_shuffle = np.float32(range_doppler_concat_shuffle)
spectrogram_concat_shuffle = np.float32(spectrogram_concat_shuffle)


In [10]:
#n_features = range_doppler_concat_shuffle.shape[1]
#n_steps = range_doppler_concat_shuffle.shape[2]
range_doppler_concat_shuffle = np.transpose(range_doppler_concat_shuffle, axes = (0,2,1,3))
spectrogram_concat_shuffle = np.transpose(spectrogram_concat_shuffle, axes = (0,2,1,3))

In [ ]:
print(range_doppler_concat.shape)
print(range_doppler_concat_label_shuffle.shape)

(171, 100, 100, 1)
(171, 1)


In [11]:
t = time.time()
# ---------- Parameters ----------------
augmentation_enable = True
normalize_inputs_enable = True
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
test_accuracy_per_run = []
f1_score_per_run = []
epoch_number = 100
batch_size = 32
dense_size = 64
dropout_prob_dense = 0.5
repeat_of_mixup = 5
number_of_repeat = 3
unit_number_of_lstm = 16
dense_unit_of_range_doppler_function = 32
dense_unit_of_spectrogram_function = 8
decoder_dense_unit = 32
for repeat_run_number in range(number_of_repeat):
  test_accuracy_per_fold = []
  f1_score_per_fold = []
  if repeat_run_number > 0:
    del range_doppler_concat_shuffle_test
    del spectrogram_concat_shuffle_test
    del range_doppler_training_data
    del range_doppler_concat_shuffle_train
    del spectrogram_concat_shuffle_train
    del spectrogram_training_data


  #print(range_doppler_concat_label_shuffle_onehotencoded.shape)
  #print(range_doppler_concat_label_shuffle_onehotencoded.shape)

  for randomlist_for_train_indx, randomlist_for_test_indx in kfold.split(range_doppler_concat_shuffle,range_doppler_concat_label_shuffle):
    gc.collect()
    K.clear_session()

    # test data
    range_doppler_concat_shuffle_test = range_doppler_concat_shuffle[randomlist_for_test_indx,:,:,:]
    spectrogram_concat_shuffle_test = spectrogram_concat_shuffle[randomlist_for_test_indx,:,:,:]
    test_label = range_doppler_concat_label_shuffle[randomlist_for_test_indx,:]
    #train data
    range_doppler_concat_shuffle_train = range_doppler_concat_shuffle[randomlist_for_train_indx,:,:,:]
    spectrogram_concat_shuffle_train = spectrogram_concat_shuffle[randomlist_for_train_indx,:,:,:]
    label_shuffle_train = spectrogram_concat_label_shuffle[randomlist_for_train_indx,:]
     # ---------------- Split Data ----------------
    (spectrogram_training_data,range_doppler_training_data,training_labels,\
     spectrogram_validation_data,range_doppler_validation_data, validation_labels)  =\
      split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,label_shuffle_train)


    # One-Hot Encode Label Train
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(training_labels)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse_output=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    training_labels_onehotencoded = onehot_encoder.fit_transform(integer_encoded)

        # One-Hot Encode Label Validation
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(validation_labels)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse_output=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    validation_labels_onehotencoded = onehot_encoder.fit_transform(integer_encoded)


       # One-Hot Encode Label Test
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(test_label)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse_output=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    test_label_onehotencoded = onehot_encoder.fit_transform(integer_encoded)





    # ---------------- Neural Network Architecture ----------------


    def lstm_encoder_network_1(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(LSTM(unit_number_of_lstm, return_sequences=True, dropout = 0.5))(input)
        x = Flatten()(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(dense_unit_of_range_doppler_function)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(dropout_prob_dense)(x)
        return Model(input, x)

    def lstm_encoder_network_2(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(LSTM(unit_number_of_lstm, return_sequences=True, dropout = 0.5))(input)
        x = Flatten()(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(dense_unit_of_spectrogram_function)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(dropout_prob_dense)(x)
        return Model(input, x)

    def decoder_for_concat(input_shape):
      input = Input(shape=input_shape)
      x = Dense(decoder_dense_unit)(input)
      x = BatchNormalization()(x)
      x = LeakyReLU()(x)
      x = Dropout(0.3)(x)
      x = Dense(dense_size)(x)
      x = LeakyReLU()(x)
      x = Dropout(dropout_prob_dense)(x)
      x = Dense(3, activation="softmax")(x)
      return Model(input, x)

    input_shape = range_doppler_concat_shuffle.shape[1:3]
    base_network_lstm = lstm_encoder_network_1(input_shape)
    range_doppler_input  = Input(shape=input_shape)
    processed_range_doppler  = base_network_lstm(range_doppler_input)

    input_shape = spectrogram_concat_shuffle_train.shape[1:3]
    base_network_lstm_2 = lstm_encoder_network_2(input_shape)
    spectrogram_input  = Input(shape=input_shape)
    processed_spectrogram  = base_network_lstm_2(spectrogram_input)

    concat_layer = Concatenate()([processed_range_doppler])

    base_decoder_network = decoder_for_concat((concat_layer.shape[1],))
    out = base_decoder_network(concat_layer)

    model = Model(inputs=[range_doppler_input], outputs=[out])
    #if repeat_run_number == 0:
     # print(base_network_lstm.summary())
     # print(base_network_lstm_2.summary())
     # print(base_decoder_network.summary())
    # ---------------- Compile and Fit ----------------
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    earlyStopping = EarlyStopping(monitor='val_loss', patience=15, verbose=0,restore_best_weights=True, mode='min')
    history = model.fit((range_doppler_training_data),(training_labels_onehotencoded),
                    epochs=epoch_number,
                    batch_size=batch_size,
                    shuffle = True,
                    callbacks=[earlyStopping],
                    validation_data = ((range_doppler_validation_data) , (validation_labels_onehotencoded)))
    test_loss, test_accuracy  = model.evaluate([range_doppler_concat_shuffle_test],\
                                               [test_label_onehotencoded],
                  batch_size=batch_size)
    gc.collect()
    # ---------------- Get Test Results ----------------
    test_accuracy_per_fold.append(test_accuracy)

    del model
  test_accuracy_per_run.append(sum(test_accuracy_per_fold)/num_folds)
  print(test_accuracy_per_run)
print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)},\
 max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))},\
 min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, \
 std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}.')
elapsed = time.time() - t
print(f'Time elapsed through all process: {"{:.3f}".format(elapsed)}, sec')

/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 211ms/step - accuracy: 0.3707 - loss: 1.4015 - val_accuracy: 0.2963 - val_loss: 1.2848
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.3624 - loss: 1.2539 - val_accuracy: 0.2963 - val_loss: 1.2762
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3274 - loss: 1.2384 - val_accuracy: 0.2963 - val_loss: 1.2477
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3751 - loss: 1.3019 - val_accuracy: 0.2963 - val_loss: 1.2316
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.3953 - loss: 1.2298 - val_accuracy: 0.2963 - val_loss: 1.2114
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.3828 - loss: 1.2234 - val_accuracy: 0.2963 - val_loss: 1.1831
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.4579 - loss: 1.1174 - val_accuracy: 0.3333 - val_loss: 1.1337
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.4730 - loss: 1.0569 - val_accuracy: 0.3333 - val_loss

/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 212ms/step - accuracy: 0.2913 - loss: 1.4063 - val_accuracy: 0.2963 - val_loss: 1.1067
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.3340 - loss: 1.2226 - val_accuracy: 0.4074 - val_loss: 1.1055
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3470 - loss: 1.2407 - val_accuracy: 0.4074 - val_loss: 1.0902
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.2435 - loss: 1.4719 - val_accuracy: 0.4074 - val_loss: 1.0691
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.2820 - loss: 1.2767 - val_accuracy: 0.4074 - val_loss: 1.0641
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.3361 - loss: 1.1477 - val_accuracy: 0.3704 - val_loss: 1.0678
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.4225 - loss: 1.1394 - val_accuracy: 0.3704 - val_loss: 1.0770
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.3106 - loss: 1.2534 - val_accuracy: 0.3704 - val_loss

/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 189ms/step - accuracy: 0.3090 - loss: 1.3404 - val_accuracy: 0.3333 - val_loss: 1.0967
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.3590 - loss: 1.2096 - val_accuracy: 0.3704 - val_loss: 1.0933
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.3225 - loss: 1.2366 - val_accuracy: 0.4074 - val_loss: 1.0802
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.4084 - loss: 1.2708 - val_accuracy: 0.4444 - val_loss: 1.0744
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3142 - loss: 1.2509 - val_accuracy: 0.4444 - val_loss: 1.0663
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3210 - loss: 1.2198 - val_accuracy: 0.4444 - val_loss: 1.0586
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3866 - loss: 1.1160 - val_accuracy: 0.4815 - val_loss: 1.0498
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3907 - loss: 1.2349 - val_accuracy: 0.4815 - val_loss

/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 203ms/step - accuracy: 0.2815 - loss: 1.4007 - val_accuracy: 0.1481 - val_loss: 1.1047
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.4116 - loss: 1.2520 - val_accuracy: 0.2963 - val_loss: 1.0951
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.2804 - loss: 1.4554 - val_accuracy: 0.2593 - val_loss: 1.0984
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.3611 - loss: 1.3569 - val_accuracy: 0.2593 - val_loss: 1.0976
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.3465 - loss: 1.2248 - val_accuracy: 0.3333 - val_loss: 1.0833
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.4251 - loss: 1.2761 - val_accuracy: 0.3333 - val_loss: 1.0778
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.4027 - loss: 1.2558 - val_accuracy: 0.4074 - val_loss: 1.0643
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.5052 - loss: 1.1974 - val_accuracy: 0.4074 - val_loss

/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 200ms/step - accuracy: 0.3980 - loss: 1.3538 - val_accuracy: 0.3333 - val_loss: 1.1880
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.3798 - loss: 1.2606 - val_accuracy: 0.3333 - val_loss: 1.2317
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3548 - loss: 1.5446 - val_accuracy: 0.3333 - val_loss: 1.2153
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.3933 - loss: 1.4582 - val_accuracy: 0.4074 - val_loss: 1.1750
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.3970 - loss: 1.2993 - val_accuracy: 0.4074 - val_loss: 1.1401
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.4116 - loss: 1.2255 - val_accuracy: 0.4074 - val_loss: 1.1108
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.3746 - loss: 1.2936 - val_accuracy: 0.4074 - val_loss: 1.0716
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.4089 - loss: 1.1950 - val_accuracy: 0.4074 - val_loss

In [12]:
print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)},\
 max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))},\
 min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, \
 std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}.')
elapsed = time.time() - t
print(f'Time elapsed through all process: {"{:.3f}".format(elapsed)}, sec')

Mean test accuracy is 0.604, max test accuracy is 0.626, min test accuracy is 0.567,  std of test accuracy is 0.027.
Time elapsed through all process: 356.646, sec
